In [2]:
# Install necessary libraries
!pip install transformers torch matplotlib seaborn tqdm scikit-learn pandas nltk rouge sentence-transformers bitsandbytes accelerate

# Import libraries
import json
import time
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer, util
from sklearn.metrics.pairwise import cosine_similarity
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge
from tqdm import tqdm
import numpy as np
import pandas as pd
import nltk
from difflib import SequenceMatcher
from accelerate import init_empty_weights, infer_auto_device_map
from transformers import BitsAndBytesConfig

# Download NLTK data for tokenization
nltk.download('punkt')

# Load the dataset from Kaggle's input directory
with open('/kaggle/input/statss/stats_problems_dataset.json', 'r') as f:  # Corrected file path
    dataset = json.load(f)

# Convert dataset to DataFrame for stratified sampling (300-400 problems)
df = pd.DataFrame(dataset)
sampled_dataset = df.groupby('difficulty', group_keys=False).apply(
    lambda x: x.sample(min(len(x), 400 // df['difficulty'].nunique()), random_state=42)
).to_dict('records')

# Load the SentenceTransformer model for embedding similarity
embedding_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Initialize ROUGE scorer
rouge = Rouge()

# Function to calculate traditional similarity (BLEU, ROUGE) and embedding similarity
def calculate_similarity(text1, text2):
    return SequenceMatcher(None, text1.lower(), text2.lower()).ratio()

def calculate_bleu_score(reference, hypothesis):
    return sentence_bleu([reference.split()], hypothesis.split())

def calculate_rouge(reference, hypothesis):
    scores = rouge.get_scores(hypothesis, reference)
    return scores[0]

def evaluate_answer(generated, correct):
    # Calculate similarity metrics
    similarity = calculate_similarity(generated, correct)
    bleu_score = calculate_bleu_score(correct, generated)
    rouge_score = calculate_rouge(correct, generated)

    # Embedding-based similarity
    generated_embedding = embedding_model.encode([generated])
    correct_embedding = embedding_model.encode([correct])
    embedding_similarity = cosine_similarity(generated_embedding, correct_embedding)[0][0]

    # Evaluation threshold
    is_correct = embedding_similarity > 0.5  # Threshold can be tuned
    return {
        "is_correct": is_correct,
        "similarity": similarity,
        "embedding_similarity": embedding_similarity,
        "bleu_score": bleu_score,
        "rouge_1": rouge_score['rouge-1']['f'],
        "rouge_2": rouge_score['rouge-2']['f'],
        "rouge_l": rouge_score['rouge-l']['f']
    }

def evaluate_model(model, tokenizer, dataset):
    results = []
    for problem in tqdm(dataset):
        prompt = f"Solve the following statistics problem step-by-step, including all necessary calculations and detailed explanations:\n\nProblem: {problem['problem']}\n\nSolution:"
        inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024).to(model.device)
        
        try:
            start_time = time.time()
            with torch.no_grad():
                outputs = model.generate(**inputs, max_new_tokens=350, num_return_sequences=1)
            generated_solution = tokenizer.decode(outputs[0], skip_special_tokens=True)
            end_time = time.time()

            evaluation_results = evaluate_answer(generated_solution, problem['solution']['conclusion'])

            results.append({
                "model_name": "unsloth/llama-3-8b-bnb-4bit",
                "topic": problem["topic"],
                "difficulty": problem["difficulty"],
                "generated_solution": generated_solution,
                "correct_solution": problem["solution"]["conclusion"],
                "time_taken": end_time - start_time,
                **evaluation_results
            })
        except Exception as e:
            print(f"Error processing problem: {e}")
    
    return results

# Step 1: Load the model using 4-bit Quantization
bnb_config = BitsAndBytesConfig(load_in_4bit=True)

# Load the LLaMA model and tokenizer
model_name = "unsloth/llama-3-8b-bnb-4bit"
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(model_name)
model.eval()

# Step 2: Evaluate model on the sampled dataset
results = evaluate_model(model, tokenizer, sampled_dataset)

# Step 3: Normalize numpy types
def normalize_types(results):
    for result in results:
        for key, value in result.items():
            if isinstance(value, np.generic):
                result[key] = value.item()
    return results

# Step 4: Normalize and save results
normalized_results = normalize_types(results)

with open('/kaggle/working/llama_results.json', 'w') as f:
    json.dump(normalized_results, f)

print("Evaluation complete. Results saved to llama_results.json")
torch.cuda.empty_cache()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 11.5 MB/s eta 0:00:0000:0100:01
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/tmp/ipykernel_36/2192305751.py:30: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  sampled_dataset = df.groupby('difficulty', group_keys=False).apply(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

...

100%|█████████▉| 319/320 [2:07:22<00:26, 26.49s/it]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

100%|██████████| 320/320 [2:07:53<00:00, 23.98s/it]

Evaluation complete. Results saved to llama_results.json
